In [1]:
import numpy as np
import random
from qutip import *
import matplotlib.pyplot as plt

# Problem 3: B92

### 3a)

In [ ]:
def Alice(Np):
    #output a list of Np rows, 2 columns
    #first col = index, second col = qubit in polarization of sent photon (in H/V basis) - random
    result = []
    for a in range(Np):
        val = random.random() #randomly assigns horizontal or diagonal basis
        if val >= .5: #if alice chooses horizontal basis
            temp_ket = basis(2,0) #|0>
        else: #if alice chooses diagonal basis
            temp_ket = (basis(2,0) + basis(2,1)).unit() #|+>
        result.append([a,temp_ket])
    return result
alice_res = Alice(8)

# Print alice data (This printing/formatting code below is made with ChatGPT's help)
print("Alice's Photons Sent:")
print("{:<10} {:<15} {:<25}".format('Photon', 'Bit Sent', 'Quantum State'))
print('-' * 50)  # Separator line
for idx, state in alice_res:
    # Determine the bit Alice sent based on the state
    if state == basis(2, 0):
        bit_sent = 0
    else:
        bit_sent = 1
    # Convert the quantum state to a string representation
    state_str = state.full().flatten()
    state_str_formatted = f"[{state_str[0]:.2f}, {state_str[1]:.2f}]"
    # Print the index, bit sent, and the quantum state
    print("{:<10} {:<15} {:<25}".format(idx, bit_sent, state_str_formatted))

Alice's Photons Sent:
Photon     Bit Sent        Quantum State            
--------------------------------------------------
0          1               [0.71+0.00j, 0.71+0.00j] 
1          0               [1.00+0.00j, 0.00+0.00j] 
2          0               [1.00+0.00j, 0.00+0.00j] 
3          1               [0.71+0.00j, 0.71+0.00j] 
4          1               [0.71+0.00j, 0.71+0.00j] 
5          1               [0.71+0.00j, 0.71+0.00j] 
6          1               [0.71+0.00j, 0.71+0.00j] 
7          1               [0.71+0.00j, 0.71+0.00j] 


### 3b)

In [ ]:
def Bob(alice_arr):
    ket_one = basis(2,1) #|1>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    result = []
    for photon in alice_arr:
        ind, ket = photon
        val2 = random.random()
        if val2 >= .5: #if Bob chooses vertical basis
            #probabilty we get a click = |<1|ket>|^2
            prob = np.abs(ket_one.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
                result.append([ind, 1])

        else: # if Bob chooses diagonal basis
            prob = np.abs(ket_minus.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
                result.append([ind, 0])
    return result




bob_results = Bob(alice_res)
# Print Bob's measurement results in columns (This printing/formatting code below is made with ChatGPT's help)
print("Bob's Measurement Results:")
print("{:<10} {:<15}".format('Photon', 'Bob Inferred Bit'))
print('-' * 25)  # Separator line
for idx, bit in bob_results:
    print("{:<10} {:<15}".format(idx, bit))

Bob's Measurement Results:
Photon     Bob Inferred Bit
-------------------------
0          1              
1          0              
3          1              
5          1              
6          1              


### 3c)

In [55]:
N = 100
alice_100 = Alice(N)
meas_results = Bob(alice_100)

#put Alice's Qobj array into bit format - makes showing success/failure more clear
alice_100_readable = []
for ind, state in alice_100:
    if state == basis(2,0): 
        alice_100_readable.append([ind, 0]) #if we are in Horizontal state she sends 0
    else:
        alice_100_readable.append([ind, 1]) #if we are in + state she sends 1

accepted = []
for ind, bob_meas in meas_results:
    accepted.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
verification_set = []
key_gen_set = []
np.random.shuffle(accepted) #randomizes order of accepted - what we will use to separate our verifcationa and key_gen sets
key_gen_set = accepted[:(len(accepted) // 2)] #first half of shuffled accepted
key_gen_set = sorted(key_gen_set, key = lambda x: x[0]) #sort by index
verification_set = accepted[(len(accepted) // 2):] #second half of shuffled accepted
verification_set = sorted(verification_set, key = lambda x: x[0]) #sort by index
private_key  = []
for i in key_gen_set:
    private_key.append(i[2]) #append Bob's measurement bit to the private key
print(private_key)

[0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]


### 3d)

In [ ]:
def Eve(alice_arr):
    ket_one = basis(2,1) #|1>
    ket_zero = basis(2,0) #|0>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    ket_plus = (basis(2,0) + basis(2,1)).unit() #|+>
    result = []
    for photon in alice_arr:
        ind, ket = photon
        val = random.random()
        if val >= .5: #if Eve chooses vertical basis
            #probabilty we get a click = |<1|psi>|^2
            prob = np.abs(ket_one.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
                result.append([ind, ket_plus]) #we recreate the plus state we know alice sent
            else:
                rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
                if rand >= 0.5:
                    choice = ket_zero
                else:
                    choice = ket_plus
                result.append([ind, choice])  
        else: #if Eve chooses diagonal basis
            prob = np.abs(ket_minus.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
                result.append([ind, ket_zero])
            else:
                rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
                if rand >= 0.5:
                    choice = ket_zero
                else:
                    choice = ket_plus
                result.append([ind, choice]) 
    return result    

N = 100
alice_100 = Alice(N)
Eve(alice_100)

[[0,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [1,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [2,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [3,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [4,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [5,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [6,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.70710678]
   [0.70710678]]],
 [7,
  Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
  Qobj data =
  [[1.]
   [0.]]],


### 3e/f)

In [ ]:
def private_key_eve(N):
    alice_output = Alice(N)
    eve_output = Eve(alice_output)
    bob_eve_output = Bob(eve_output)

    alice_100_readable = []
    for ind, state in alice_output:
        if state == basis(2,0): 
            alice_100_readable.append([ind, 0]) #if we are in Horizontal state she sends 0
        else:
            alice_100_readable.append([ind, 1]) #if we are in + state she sends 1


    accepted = []
    for ind, bob_meas in bob_eve_output:
        accepted.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
    

    verification_set = []
    key_gen_set = []
    np.random.shuffle(accepted) #randomizes order of accepted - what we will use to separate our verifcationa and key_gen sets
    key_gen_set = accepted[:(len(accepted) // 2)] #first half of shuffled accepted
    key_gen_set = sorted(key_gen_set, key = lambda x: x[0]) #sort by index
    verification_set = accepted[(len(accepted) // 2):] #second half of shuffled accepted
    verification_set = sorted(verification_set, key = lambda x: x[0]) #sort by index

    errors = sum(1 for ind, a_bit, b_bit in verification_set if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
    error_rate = errors/len(verification_set)

    private_key  = []
    actual_private_key = []
    for i in key_gen_set:
        private_key.append(i[2]) #append Bob's measurement bit to the private key
        actual_private_key.append(i[1]) #append the actual bit alice sent that bob should have measured to the actual pribate key
    print(f"Final private key: {private_key}")
    print(f"Actual private key: {actual_private_key}")
    return error_rate


print("Error Rate:", private_key_eve(1000))


Final private key: [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Actual private key: [0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,